# Big Data Analytics: NYC Crashes

## Einleitung

### Rahmenbedingungen des Projekts

### Aufbau der Datenpipeline

#### Installation der benötigten Docker-Container

### Datenquelle

### Analyseziele

### Installation der Python Packages

In [ ]:
pip install kafka-python pymongo

### Importieren benötigter Module

In [1]:
from kafka import KafkaProducer, KafkaConsumer
from pymongo import MongoClient
import datetime as dt
import requests
import os
from pathlib import Path

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Herunterladen der Datensätze (H)
Zum herunterladen der Datensätze werden die Daten von der Datenquelle mithilfe des nachfolgenden Python-Scripts heruntergeladen. Resultat sind drei `.csv`-Dateien die die Unfalldaten zeilenweise enthalten.

In [2]:
if not os.path.exists('data/'):
    os.mkdir('data')

for file_name, download_url in [
    ('crashes.csv', 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'),
    ('vehicles.csv', 'https://data.cityofnewyork.us/api/views/bm4k-52h4/rows.csv?accessType=DOWNLOAD'),
    ('persons.csv', 'https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=DOWNLOAD'),
]:
    if not os.path.isfile(fp:= (Path('data') / file_name)):
        with open(fp, 'wb') as crash_file:
            crash_file.write(requests.get(download_url).content)

In [3]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

dataset = open('Motor_Vehicle_Collisions_-_Crashes.csv', encoding='utf-8')
rows = dataset.readlines()[1:]

for i, row in enumerate(rows):
        producer.send('nyc_crashes', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

In [8]:
client = MongoClient("localhost:27017")

crashes = client['nyc_crashes']['crashes']
crashes.delete_many({})

consumer = KafkaConsumer('nyc_crashes', bootstrap_servers=['localhost:9092'], auto_offset_reset="earliest")

count = 0

for msg in consumer: 
    count += 1
    print('Received new message: %s' % count)
    values = msg.value.decode('utf-8').split(',')
    
    crashes.insert_one({
        'CRASH_DATE': values[0],
        'CRASH_TIME': values[1],
        'BOROUGH': values[2],
        'ZIP_CODE': int(values[3]) if values[3] else None,
        'LATITUDE': values[4],
        'LONGITUDE': values[5],
        'LOCATION': values[6],
        'ON_STREET_NAME': values[7],
        'CROSS_STREET_NAME': values[8],
        'OFF_STREET_NAME': values[9],
        'NUMBER_OF_PERSONS_INJURED': int(values[10]) if values[10] else None,
        'NUMBER_OF_PERSONS_KILLED': int(values[11]) if values[11] else None,
        'NUMBER_OF_PEDESTRIANS_INJURED': int(values[12]) if values[12] else None,
        'NUMBER_OF_PEDESTRIANS_KILLED': int(values[13]) if values[13] else None,
        'NUMBER_OF_CYCLISTS_INJURED': int(values[14]) if values[14] else None,
        'NUMBER_OF_CYCLISTS_KILLED': int(values[15]) if values[15] else None,
        'NUMBER_OF_MOTORIST_INJURED': int(values[16]) if values[16] else None,
        'NUMBER_OF_MOTORIST_KILLED': (values[17]),
        'CONTRIBUTING_FACTOR_VEHICLE_1': values[18],
        'CONTRIBUTING_FACTOR_VEHICLE_2': values[19],
        'CONTRIBUTING_FACTOR_VEHICLE_3': values[20],
        'CONTRIBUTING_FACTOR_VEHICLE_4': values[21],
        'CONTRIBUTING_FACTOR_VEHICLE_5': values[22],
        'COLLISION_ID': int(values[23]) if values[23] else None,
        'VEHICLE_TYPE_CODE_1': values[24],
        'VEHICLE_TYPE_CODE_2': values[25],
        'VEHICLE_TYPE_CODE_3': values[26],
        'VEHICLE_TYPE_CODE_4': values[27],
        'VEHICLE_TYPE_CODE_5': values[28]
    })

Received new message: 1
Received new message: 2
Received new message: 3
Received new message: 4
Received new message: 5
Received new message: 6
Received new message: 7
Received new message: 8
Received new message: 9
Received new message: 10
Received new message: 11
Received new message: 12
Received new message: 13
Received new message: 14
Received new message: 15
Received new message: 16
Received new message: 17
Received new message: 18
Received new message: 19
Received new message: 20
Received new message: 21
Received new message: 22
Received new message: 23
Received new message: 24
Received new message: 25
Received new message: 26
Received new message: 27
Received new message: 28
Received new message: 29
Received new message: 30
Received new message: 31
Received new message: 32
Received new message: 33
Received new message: 34
Received new message: 35
Received new message: 36
Received new message: 37
Received new message: 38
Received new message: 39
Received new message: 40
Received 

ValueError: invalid literal for int() with base 10: ' DRIVEWAY            "'

In [ ]:


producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

for i in range(1, 13):
    if i < 10:
        i = "0%s" % i
    dataset = open('./data/2019/yellow-cabs-2019-%s.csv' % i, encoding='utf-8')
    rows = dataset.readlines()[1:]

    for i, row in enumerate(rows):
        producer.send('yellow-cabs', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

for i in range(1, 7):
    if i < 10:
        i = "0%s" % i
    dataset = open('./data/2019/yellow-cabs-2020-%s.csv' % i, encoding='utf-8')
    rows = dataset.readlines()[1:]

    for i, row in enumerate(rows):
        producer.send('yellow-cabs', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

In [ ]:


consumer = KafkaConsumer('yellow-cabs', bootstrap_servers=[""])

client = MongoClient("")

yellow_collection = client['datawarehouse']['bg-yellowcabs']
yellow_collection.delete_many({})

count = 0

for msg in consumer: 
    count += 1
    print('Received new message: %s' % count)
    values = msg.value.decode('utf-8').split(',')
    
    yellow_collection.insert_one({
        'pickup_datetime': dt.datetime.strptime(values[1], "%Y-%m-%d %H:%M:%S"),
        'dropoff_datetime': dt.datetime.strptime(values[2], "%Y-%m-%d %H:%M:%S"),
        'passenger_count': int(values[3]),
        'trip_distance': float(values[4]),
        'PULocationID': values[5],
        'DOLocationID': values[6],
        'payment_type': int(values[9]),
        'fare_amount': float(values[10]),
        'tip_amount': float(values[15]),
        'total_amount': float(values[16])
    })